In [1]:
import plotly.graph_objects as go
from plotly.colors import n_colors
import numpy as np
import pandas as pd

In [3]:
itinerarios_bases=pd.read_parquet('../Data/Itinerarios/itinerarios_bases3.parquet')

In [4]:
itinerarios_bases.dtypes

user_type                                  int8
travel_time                               int32
idunplug_station                          int16
ageRange                                   int8
idplug_station                            int16
unplug_hourTime             datetime64[ns, UTC]
status                                   object
Origen_destino                           object
Número_Salida                             int16
Distrito_Salida                          object
Barrio_Salida                            object
Número de Plazas_Salida                 float32
Latitud_Salida                          float32
Longitud_Salida                         float32
Número_Llegada                            int16
Distrito_Llegada                         object
Barrio_Llegada                           object
Número de Plazas_Llegada                float32
Latitud_Llegada                         float32
Longitud_Llegada                        float32
name_Salida                             

In [15]:
itinerarios_bases["lost"] = np.where(itinerarios_bases["status"]=="long_rental",1,0)
itinerarios_bases["change_bike"] = np.where(itinerarios_bases["status"]=="change_bike",1,0)
itinerarios_bases["successful"] = np.where(itinerarios_bases["status"]=="short_rental",1,0)
bases = itinerarios_bases.groupby(["idunplug_station", "name_Salida", "address_Salida", "Distrito_Salida", "Barrio_Salida", "Número de Plazas_Salida"], as_index=False).agg({"lost":"sum","change_bike":"sum", "successful":"sum"})
bases.head()

,idunplug_station,name_Salida,address_Salida,Distrito_Salida,Barrio_Salida,Número de Plazas_Salida,lost,change_bike,successful
0,1,Puerta del Sol,Puerta del Sol nº 1,01 CENTRO,01-06 SOL,60.0,1146,3,9527
1,2,Miguel Moya,Calle Miguel Moya nº 1,01 CENTRO,01-05 UNIVERSIDAD,24.0,253,3,2247
2,3,Plaza Conde Suchil,Plaza del Conde Suchil nº 2-4,07 CHAMBERÍ,07-02 ARAPILES,18.0,1387,7,9128
3,4,Malasaña,Calle Manuela Malasaña nº 5,01 CENTRO,01-05 UNIVERSIDAD,24.0,628,6,6648
4,5,Fuencarral,Calle Fuencarral nº 108,01 CENTRO,01-04 JUSTICIA,27.0,840,26,5593


In [19]:
fig = go.Figure(data=[go.Table(
  header=dict(
    values=['<b>Id</b>','<b>Nombre</b>', '<b>Dirección</b>', '<b>Distrito</b>', '<b>Barrio</b>', '<b>Nº Bases</b>', '<b>Ratio Perdidas %</b>', '<b>Ratio defectuosas %</b>',],
    line_color='white', fill_color='#18bc9c', font=dict(color="white"),
    align='center',
  ),
  cells=dict(
    values=[bases["idunplug_station"], bases["name_Salida"], bases["address_Salida"], bases["Distrito_Salida"], bases["Barrio_Salida"], round(bases["Número de Plazas_Salida"]), round(bases["lost"]/bases["successful"]*100), round(bases["change_bike"]/bases["successful"]*100, 2)],
    align='center', fill_color='white', font=dict(color='#18bc9c', size=11)
    ))
])

fig.show()